In [1]:
!pip install fuzzywuzzy
!pip install openpyxl
!pip install rapidfuzz

In [2]:
import pandas as pd
import numpy as np
from numpy import mean,std
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,Normalizer,MinMaxScaler
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sklearn.metrics as metrics


import plotly.express as px


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#act=pd.read_csv('Lapsed paid members Sep 2024.csv')
act=pd.read_excel('fuzzy_data/Lapsed paid members Dec 2024 (1).xlsx')
#lap=pd.read_csv('New Basic Sep 2024.csv').dropna()
lap=pd.read_excel('fuzzy_data/New Basics Dec 2024.xlsx').dropna()
#lap=pd.read_excel('Truly Lapsed Premiers with purchase volume greater or equal 50 with pertinent details Apr 23_Mar24.xlsx')

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

/opt/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,65713,MIRANDA KIMBLE NEWELL,8046502266,MIRANDA KIMBLE NEWELL,250 Woodberry Dr,HARTFIELD,VA,USA,MIRANDA KIMBLE NEWELL,250 WOODBERRY,MIRANDA KIMBLE NEWELL 250 WOODBERRY,USA VA HARTFIELD 250 WOODBERRY,MIRANDA KIMBLE NEWELL MIRANDA KIMBLE NEWELL US...,MIRANDA KIMBLE NEWELL MIRANDA KIMBLE NEWELL,1
1,89291,FELICIA KATRINA MATEN,5049390785,FELICIA KATRINA MATEN,1026 Hendee,NEW ORLEANS,LA,USA,FELICIA KATRINA MATEN,1026 HENDEE,FELICIA KATRINA MATEN 1026 HENDEE,USA LA NEW ORLEANS 1026 HENDEE,FELICIA KATRINA MATEN FELICIA KATRINA MATEN US...,FELICIA KATRINA MATEN FELICIA KATRINA MATEN,1
2,88221,BRAYDEN GARY AITKEN,8605340809,BRAYDEN GARY AITKEN,102 Timberlane,CANTON,CT,USA,BRAYDEN GARY AITKEN,102 TIMBERLANE,BRAYDEN GARY AITKEN 102 TIMBERLANE,USA CT CANTON 102 TIMBERLANE,BRAYDEN GARY AITKEN BRAYDEN GARY AITKEN USA CT...,BRAYDEN GARY AITKEN BRAYDEN GARY AITKEN,1
3,26307,EDGAR R BERZUNZA PACHECO,3604337298,EDGAR R BERZUNZA PACHECO,1919 w 34th st #e9,VANCOUVER,WA,USA,EDGAR R BERZUNZA PACHECO,1919 W 34TH #E9,EDGAR R BERZUNZA PACHECO 1919 W 34TH #E9,USA WA VANCOUVER 1919 W 34TH #E9,EDGAR R BERZUNZA PACHECO EDGAR R BERZUNZA PACH...,EDGAR R BERZUNZA PACHECO EDGAR R BERZUNZA PACHECO,1
4,910497,LUIS FELIPE LETUSET CABRERA,8325638907,LUIS FELIPE LETUSET CABRERA,3900 CITY AVE APT A610,PHILADELPHIA,PA,USA,LUIS FELIPE LETUSET CABRERA,3900 CITY APT A610,LUIS FELIPE LETUSET CABRERA 3900 CITY APT A610,USA PA PHILADELPHIA 3900 CITY APT A610,LUIS FELIPE LETUSET CABRERA LUIS FELIPE LETUSE...,LUIS FELIPE LETUSET CABRERA LUIS FELIPE LETUSE...,1


In [3]:
lap.head(2)
act.head(2)
lap.tail(2)
act.tail(2)

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,65713,MIRANDA KIMBLE NEWELL,8046502266,MIRANDA KIMBLE NEWELL,250 Woodberry Dr,HARTFIELD,VA,USA,MIRANDA KIMBLE NEWELL,250 WOODBERRY,MIRANDA KIMBLE NEWELL 250 WOODBERRY,USA VA HARTFIELD 250 WOODBERRY,MIRANDA KIMBLE NEWELL MIRANDA KIMBLE NEWELL US...,MIRANDA KIMBLE NEWELL MIRANDA KIMBLE NEWELL,1
1,89291,FELICIA KATRINA MATEN,5049390785,FELICIA KATRINA MATEN,1026 Hendee,NEW ORLEANS,LA,USA,FELICIA KATRINA MATEN,1026 HENDEE,FELICIA KATRINA MATEN 1026 HENDEE,USA LA NEW ORLEANS 1026 HENDEE,FELICIA KATRINA MATEN FELICIA KATRINA MATEN US...,FELICIA KATRINA MATEN FELICIA KATRINA MATEN,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,256260,BRANDON BYRON,7013735069,BRANDON BYRON,3380 Maple Leaf Loop S,FARGO,ND,USA,BRANDON BYRON,3380 MAPLE LEAF LOOP S,BRANDON BYRON 3380 MAPLE LEAF LOOP S,USA ND FARGO 3380 MAPLE LEAF LOOP S,BRANDON BYRON BRANDON BYRON USA ND FARGO 3380 ...,BRANDON BYRON BRANDON BYRON,0
1,253635,JOSE J LOPEZ HERRERA,9545940965,JOSE J LOPEZ HERRERA,210 CENTER ST,BREWER,ME,USA,JOSE J LOPEZ HERRERA,210 CENTER,JOSE J LOPEZ HERRERA 210 CENTER,USA ME BREWER 210 CENTER,JOSE J LOPEZ HERRERA JOSE J LOPEZ HERRERA USA ...,JOSE J LOPEZ HERRERA JOSE J LOPEZ HERRERA,0


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
9947,900834,OLAKUNLE B OYEWOLE,5162630782,OLAKUNLE B OYEWOLE,135 Adler Road,WEST COLUMBIA,SC,USA,OLAKUNLE B OYEWOLE,135 ADLER,OLAKUNLE B OYEWOLE 135 ADLER,USA SC WEST COLUMBIA 135 ADLER,OLAKUNLE B OYEWOLE OLAKUNLE B OYEWOLE USA SC W...,OLAKUNLE B OYEWOLE OLAKUNLE B OYEWOLE,1
9948,903099,PHILLIP WILLIAM JACOBY,7852148125,PHILLIP WILLIAM JACOBY,3520 W 22ND ST APT G4 G4,LAWRENCE,KS,USA,PHILLIP WILLIAM JACOBY,3520 W 22ND APT G4 G4,PHILLIP WILLIAM JACOBY 3520 W 22ND APT G4 G4,USA KS LAWRENCE 3520 W 22ND APT G4 G4,PHILLIP WILLIAM JACOBY PHILLIP WILLIAM JACOBY ...,PHILLIP WILLIAM JACOBY PHILLIP WILLIAM JACOBY,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
330083,418130,TIMOTHY THOMAS,2404165075,ROYAL-T AUTO GROUP LLC,101 Skyline Drive #1 W421,ARLINGTON,WI,USA,ROYAL-T GROUP,101 SKYLINE #1 W421,ROYAL-T GROUP 101 SKYLINE #1 W421,USA WI ARLINGTON 101 SKYLINE #1 W421,TIMOTHY THOMAS ROYAL-T GROUP USA WI ARLINGTO...,TIMOTHY THOMAS ROYAL-T GROUP,0
330084,275610,JEROME PACE SR,7737427786,TOKYO AUTO GROUP LLC,101 SKYLINE DR # 1 W431,ARLINGTON,WI,USA,TOKYO GROUP,101 SKYLINE # 1 W431,TOKYO GROUP 101 SKYLINE # 1 W431,USA WI ARLINGTON 101 SKYLINE # 1 W431,JEROME PACE SR TOKYO GROUP USA WI ARLINGTON ...,JEROME PACE SR TOKYO GROUP,0


In [4]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('fuzzy_data/New Basics Dec 2024 Lapsed Paid Members Dec 2024.csv')

--- 24031.382384061813 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,65713,MIRANDA KIMBLE NEWELL,8046502266,MIRANDA KIMBLE NEWELL,250 Woodberry Dr,MIRANDA KIMBLE NEWELL MIRANDA KIMBLE NEWELL US...,1,778669,MICHAEL MIRANDA,7166323900,MIRANDA AUTO INC,8925 SHERIDAN DR,MICHAEL MIRANDA MIRANDA USA NY CLARENCE 8925...,0,53.608247,63.636364,47.457627,63.636364,100.000000,hier 4
1,89291,FELICIA KATRINA MATEN,5049390785,FELICIA KATRINA MATEN,1026 Hendee,FELICIA KATRINA MATEN FELICIA KATRINA MATEN US...,1,262323,TRICHELL SORRELL,5044539574,TRICHELL SORRELL,3427 Huntlee Drive,TRICHELL SORRELL TRICHELL SORRELL USA LA NEW O...,0,60.000000,37.837838,81.967213,37.837838,37.837838,hier 4
2,89291,FELICIA KATRINA MATEN,5049390785,FELICIA KATRINA MATEN,1026 Hendee,FELICIA KATRINA MATEN FELICIA KATRINA MATEN US...,1,214258,ANTONIO GREEN,5042025657,RUN & DRIVE AUTOSPORTS,1310 REYNES ST,ANTONIO GREEN RUN & DRIVE SPORTS USA LA NEW OR...,0,59.130435,30.188679,83.333333,35.294118,30.769231,hier 4
3,89291,FELICIA KATRINA MATEN,5049390785,FELICIA KATRINA MATEN,1026 Hendee,FELICIA KATRINA MATEN FELICIA KATRINA MATEN US...,1,572479,BRAYAN OYUELA FLORES,5049829648,BRAYAN OYUELA FLORES,3816 Orleans Ave,BRAYAN OYUELA FLORES BRAYAN OYUELA FLORES USA ...,0,58.333333,24.390244,87.804878,24.390244,24.390244,hier 4
4,89291,FELICIA KATRINA MATEN,5049390785,FELICIA KATRINA MATEN,1026 Hendee,FELICIA KATRINA MATEN FELICIA KATRINA MATEN US...,1,170451,MELBA GIBSON,5044152335,MELBA GIBSON,1308 Reynes St,MELBA GIBSON MELBA GIBSON USA LA NEW ORLEANS 1...,0,61.052632,30.303030,83.333333,30.303030,30.303030,hier 4


In [ ]:
# import pandas as pd
# import numpy as np
# x=pd.read_csv('Lapsed Paid Members 07162024_New_Basics_june24.csv')
# x.tail()

# 2

In [9]:


#act=pd.read_csv('active paid members Sep 2024.csv')
act=pd.read_excel('fuzzy_data/active paid members Dec 2024.xlsx')
#lap=pd.read_csv('Lapsed Basics Sep 2024.csv').dropna()
lap=pd.read_excel('fuzzy_data/Lapsed Basic Dec 2024.xlsx').dropna().rename(columns={'mbr_nbr_1': 'mbr_nbr'})
#lap=pd.read_excel('Truly Lapsed Premiers with purchase volume greater or equal 50 with pertinent details Apr 23_Mar24.xlsx')

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,300435,JOSHUA AINSWORTH,2094715089,JOSHUA AINSWORTH,457-095 US Highway 395,MILFORD,CA,USA,JOSHUA AINSWORTH,457-095 US 395,JOSHUA AINSWORTH 457-095 US 395,USA CA MILFORD 457-095 US 395,JOSHUA AINSWORTH JOSHUA AINSWORTH USA CA MILFO...,JOSHUA AINSWORTH JOSHUA AINSWORTH,1
1,250759,ETHAN LESTER,4437637431,ETHAN LESTER,2785 Wade Trail,TALLAHASSEE,FL,USA,ETHAN LESTER,2785 WADE TRAIL,ETHAN LESTER 2785 WADE TRAIL,USA FL TALLAHASSEE 2785 WADE TRAIL,ETHAN LESTER ETHAN LESTER USA FL TALLAHASSEE 2...,ETHAN LESTER ETHAN LESTER,1
2,305303,VICTORIA CLEMONS,3868524377,APPLIANCE WHISPERER LLC,459 Knox McRae dr,TITUSVILLE,FL,USA,APPLIANCE WHISPERER,459 KNOX MCRAE,APPLIANCE WHISPERER 459 KNOX MCRAE,USA FL TITUSVILLE 459 KNOX MCRAE,VICTORIA CLEMONS APPLIANCE WHISPERER USA FL T...,VICTORIA CLEMONS APPLIANCE WHISPERER,1
3,311548,YUNES ALAMRI,5172271867,YUNES ALAMRI,124 s clay st,COLDWATER,MI,USA,YUNES ALAMRI,124 S CLAY,YUNES ALAMRI 124 S CLAY,USA MI COLDWATER 124 S CLAY,YUNES ALAMRI YUNES ALAMRI USA MI COLDWATER 124...,YUNES ALAMRI YUNES ALAMRI,1
4,859859,WALID HAMADAN,3194007945,WALID HAMADAN,3920 FINLEY RD,IRVING,TX,USA,WALID HAMADAN,3920 FINLEY,WALID HAMADAN 3920 FINLEY,USA TX IRVING 3920 FINLEY,WALID HAMADAN WALID HAMADAN USA TX IRVING 3920...,WALID HAMADAN WALID HAMADAN,1


In [10]:
lap.head(2)
act.head(2)
lap.tail(2)
act.tail(2)

,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,300435,JOSHUA AINSWORTH,2094715089,JOSHUA AINSWORTH,457-095 US Highway 395,MILFORD,CA,USA,JOSHUA AINSWORTH,457-095 US 395,JOSHUA AINSWORTH 457-095 US 395,USA CA MILFORD 457-095 US 395,JOSHUA AINSWORTH JOSHUA AINSWORTH USA CA MILFO...,JOSHUA AINSWORTH JOSHUA AINSWORTH,1
1,250759,ETHAN LESTER,4437637431,ETHAN LESTER,2785 Wade Trail,TALLAHASSEE,FL,USA,ETHAN LESTER,2785 WADE TRAIL,ETHAN LESTER 2785 WADE TRAIL,USA FL TALLAHASSEE 2785 WADE TRAIL,ETHAN LESTER ETHAN LESTER USA FL TALLAHASSEE 2...,ETHAN LESTER ETHAN LESTER,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
0,613183,SALIM ALHWISHEM ALHWISHEM,97304824,SALIM ALHWISHEM ALHWISHEM,Japar Al Ahamad Str114,JAPAR ALAHMAD,KW,KWT,SALIM ALHWISHEM ALHWISHEM,JAPAR AL AHAMAD R114,SALIM ALHWISHEM ALHWISHEM JAPAR AL AHAMAD R114,KWT KW JAPAR ALAHMAD JAPAR AL AHAMAD R114,SALIM ALHWISHEM ALHWISHEM SALIM ALHWISHEM ALHW...,SALIM ALHWISHEM ALHWISHEM SALIM ALHWISHEM ALHW...,0
1,302146,ROBERTO LUIS NIEVES DEJESUS,9789160796,ROBERTO LUIS NIEVES DEJESUS,48 overledge dr,DERRY,NH,USA,ROBERTO LUIS NIEVES DEJESUS,48 OVERLEDGE,ROBERTO LUIS NIEVES DEJESUS 48 OVERLEDGE,USA NH DERRY 48 OVERLEDGE,ROBERTO LUIS NIEVES DEJESUS ROBERTO LUIS NIEVE...,ROBERTO LUIS NIEVES DEJESUS ROBERTO LUIS NIEVE...,0


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
10897,84661,NELSON FLORES,8324895884,NELSON FLORES,13047 Meadowline DR,HOUSTON,TX,USA,NELSON FLORES,13047 MEADOWLINE,NELSON FLORES 13047 MEADOWLINE,USA TX HOUSTON 13047 MEADOWLINE,NELSON FLORES NELSON FLORES USA TX HOUSTON 130...,NELSON FLORES NELSON FLORES,1
10898,273701,JESUS CRUZ,2815076689,JESUS CRUZ,226 Whitehall Dr,HOUSTON,TX,USA,JESUS CRUZ,226 WHITEHALL,JESUS CRUZ 226 WHITEHALL,USA TX HOUSTON 226 WHITEHALL,JESUS CRUZ JESUS CRUZ USA TX HOUSTON 226 WHITE...,JESUS CRUZ JESUS CRUZ,1


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,mbr_city,mbr_state,mbr_country,new_name,new_add,name_add,complete_add,name_company_add,name_company,lapsed_flag
315125,933470,BRIAN HABLE,6513298045,BRIAN HABLE,631 Highway 65,ROBERTS,WI,USA,BRIAN HABLE,631 65,BRIAN HABLE 631 65,USA WI ROBERTS 631 65,BRIAN HABLE BRIAN HABLE USA WI ROBERTS 631 65,BRIAN HABLE BRIAN HABLE,0
315126,944029,FREDY A CORRO RUIZ,2624244134,MONARCAS AUTO LLC,5610 W NATIONAL AVE,WEST ALLIS,WI,USA,MONARCAS,5610 W NATIONAL,MONARCAS 5610 W NATIONAL,USA WI WEST ALLIS 5610 W NATIONAL,FREDY A CORRO RUIZ MONARCAS USA WI WEST ALL...,FREDY A CORRO RUIZ MONARCAS,0


In [11]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('fuzzy_data/Lapsed Basics Dec 2024 Active Paid Members Dec 2024 Fuzzy.csv')

--- 19841.929333925247 seconds ---


,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,mbr_nbr,mbr_name,mbr_phone1,mbr_company,mbr_addr,name_company_add,lapsed_flag,overall_score,name_comp_score,add_score,name_score,comp_score,rank
0,300435,JOSHUA AINSWORTH,2094715089,JOSHUA AINSWORTH,457-095 US Highway 395,JOSHUA AINSWORTH JOSHUA AINSWORTH USA CA MILFO...,1,473136,JOSHUA WHITWORTH,2567950132,JOSHUA WHITWORTH,24314 Hays Mill Rd,JOSHUA WHITWORTH JOSHUA WHITWORTH USA AL ELKMO...,0,53.763441,81.250000,44.067797,81.250000,81.250000,hier 4
1,300435,JOSHUA AINSWORTH,2094715089,JOSHUA AINSWORTH,457-095 US Highway 395,JOSHUA AINSWORTH JOSHUA AINSWORTH USA CA MILFO...,1,572934,JOSHUA CARR,7208386070,JOSHUA CARR,7385 E 9TH AVE,JOSHUA CARR JOSHUA R USA CO DENVER 7385 E 9TH,0,57.142857,63.157895,49.056604,70.588235,85.714286,hier 4
2,300435,JOSHUA AINSWORTH,2094715089,JOSHUA AINSWORTH,457-095 US Highway 395,JOSHUA AINSWORTH JOSHUA AINSWORTH USA CA MILFO...,1,121844,JOSHUA CARL,9123227067,JOSHUA CARL,124 Walnut Way,JOSHUA CARL JOSHUA L USA GA KINGSLAND 124 WALN...,0,50.549451,63.157895,43.333333,70.588235,85.714286,hier 4
3,300435,JOSHUA AINSWORTH,2094715089,JOSHUA AINSWORTH,457-095 US Highway 395,JOSHUA AINSWORTH JOSHUA AINSWORTH USA CA MILFO...,1,493217,JOHN AINSWORTH,6019417133,JOHN AINSWORTH,1030 E LYNN STREET,JOHN AINSWORTH JOHN AINSWORTH USA MO SPRINGFIE...,0,57.142857,86.666667,40.677966,86.666667,86.666667,hier 4
4,250759,ETHAN LESTER,4437637431,ETHAN LESTER,2785 Wade Trail,ETHAN LESTER ETHAN LESTER USA FL TALLAHASSEE 2...,1,100216,COPART YARD-,5555555555,COPART YARD-,COPART,COPART YARD- CO YARD- USA FL TALLAHASSEE COPART,0,69.230769,22.222222,83.720930,25.000000,20.000000,hier 4


# 3

In [ ]:

#act=pd.read_csv('active paid members Sep 2024.csv')
act=pd.read_excel('fuzzy_data/active paid members Dec 2024.xlsx')
#lap=pd.read_csv('New Basic Sep 2024.csv').dropna()
lap=pd.read_excel('fuzzy_data/New Basics Dec 2024.xlsx').dropna()
#lap=pd.read_excel('Anant - Truly new members promo code analysis.xlsx')

lap['new_name']=lap['mbr_company'].str.upper()
act['new_name']=act['mbr_company'].str.upper()
for x in ['AUTOMOTIVE',"SALES", "AUTO", "SALE","LLC","MOTORS","PART","LTD","INC","CAR","ENTERPRISES","ENTERPRISE","AUTO SALE","SALES LLC"]:
    lap['new_name']=lap['new_name'].str.replace(x, '')
    act['new_name']=act['new_name'].str.replace(x, '' )    

    
lap['new_add']=lap['mbr_addr'].str.upper()
act['new_add']=act['mbr_addr'].str.upper()
lap['mbr_city']=lap['mbr_city'].str.upper()
act['mbr_city']=act['mbr_city'].str.upper()

lap['mbr_name']=lap['mbr_name'].str.upper()
act['mbr_name']=act['mbr_name'].str.upper()

lap['mbr_company']=lap['mbr_company'].str.upper()
act['mbr_company']=act['mbr_company'].str.upper()

for x in ['STREET','AVENUE','DRIVE','ROAD','AVE','RD','ST','UNIT','DR','BLVD','HIGHWAY']:
    lap['new_add']=lap['new_add'].str.replace(x, '')
    act['new_add']=act['new_add'].str.replace(x, '' )  

lap['name_add']=lap['new_name']+' '+lap['new_add']
act['name_add']=act['new_name']+' '+act['new_add']

lap['complete_add']=lap['mbr_country']+' '+lap['mbr_state']+' '+lap['mbr_city']+' '+lap['new_add']
act['complete_add']=act['mbr_country']+' '+act['mbr_state']+' '+act['mbr_city']+' '+act['new_add']


lap['name_company_add']=lap['mbr_name']+' '+lap['new_name']+' '+lap['complete_add']
act['name_company_add']=act['mbr_name']+' '+act['new_name']+' '+act['complete_add']


lap['name_company']=lap['mbr_name']+' '+lap['new_name']
act['name_company']=act['mbr_name']+' '+act['new_name']


lap['lapsed_flag']=1
act['lapsed_flag']=0
df=pd.concat([lap,act],axis=0)
df.head()

In [ ]:
lap.head(2)
act.head(2)
lap.tail(2)
act.tail(2)

In [ ]:
from rapidfuzz import fuzz
import pandas as pd
import time
from joblib import Parallel, delayed

# Sample data for lapx and actx
lapx = lap.copy()
actx = act.copy()
start_time = time.time()
def process_row(i):
    row = lapx.iloc[i]
    name = row['mbr_name']
    comp = row['mbr_company']
    addr = row['complete_add']
    no = row['mbr_phone1']
    nbr = row['mbr_nbr']
    
    temp = actx.copy()
    results = []
    
    for j in range(temp.shape[0]):
        temp_row = temp.iloc[j]
        overall_score= fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add'])
        add_score=fuzz.token_set_ratio(row['complete_add'], temp_row['complete_add'])
        name_comp_score= fuzz.token_set_ratio(row['name_company'], temp_row['name_company'])
        name_score= fuzz.token_set_ratio(row['mbr_name'], temp_row['mbr_name'])
        comp_score= fuzz.token_set_ratio(row['new_name'], temp_row['new_name'])
        
        
        
        add_row = [
            row['mbr_nbr'], row['mbr_name'], row['mbr_phone1'], row['mbr_company'], row['mbr_addr'], 
            row['name_company_add'], row['lapsed_flag'],
            temp_row['mbr_nbr'], temp_row['mbr_name'], temp_row['mbr_phone1'], temp_row['mbr_company'], 
            temp_row['mbr_addr'], temp_row['name_company_add'], temp_row['lapsed_flag'],
            overall_score,name_comp_score,add_score,name_score,comp_score
        ]
        
        if ((name == temp_row['mbr_name']) and (comp == temp_row['mbr_company']) and
            (addr == temp_row['complete_add']) and (no == temp_row['mbr_phone1']) and (nbr != temp_row['mbr_nbr'])):
            add_row.append('hier 1')
            results.append(add_row)
            
        elif (((name == temp_row['mbr_name']) + (comp == temp_row['mbr_company']) +
               (addr == temp_row['complete_add']) + (no == temp_row['mbr_phone1'])) > 2) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 2')
            results.append(add_row)
            
        elif (fuzz.token_set_ratio(row['name_company_add'], temp_row['name_company_add']) > 70) and (nbr != temp_row['mbr_nbr']):
            add_row.append('hier 3')
            results.append(add_row)
            
        elif ((name_score>80) |  (add_score>80) |  (comp_score>80)):
            add_row.append('hier 4')
            results.append(add_row)

            
    return results

# Run the parallel processing
results = Parallel(n_jobs=-1)(delayed(process_row)(i) for i in range(lapx.shape[0]))

# Flatten the results list
mat = [item for sublist in results for item in sublist]


# Display the result

print("--- %s seconds ---" % (time.time() - start_time))
mat=pd.DataFrame(mat,columns=['mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'mbr_nbr', 'mbr_name', 'mbr_phone1', 'mbr_company', 'mbr_addr', 'name_company_add', 'lapsed_flag',
                                 'overall_score','name_comp_score','add_score','name_score','comp_score', 'rank'])
mat.head()
mat.to_csv('fuzzy_data/New Basics Dec 2024 Active Paid Members Dec 2024 Fuzzy.csv')